# Part 4: AST Debugging Tools

This notebook covers essential debugging and introspection tools for working with Python's AST module.

## Prerequisites

This is part 4 of the AST guide series. Before proceeding, make sure you're familiar with:
- Part 1: AST Core Concepts - Understanding nodes and the visitor pattern
- Part 2: Essential Node Types - Function, class, call, and assignment nodes
- Part 3: Practical Techniques - Scope tracking and name resolution

## What You'll Learn

- How to use `ast.dump()` for debugging AST structures
- Defensive programming with node type checking
- Safe navigation of AST node hierarchies
- Common debugging patterns and helper functions

## Import Required Modules

In [1]:
import ast
from typing import Optional

## 17. Using ast.dump() for Debugging

The `ast.dump()` function is your most powerful debugging tool when working with AST. It shows you the exact structure of any AST node, including all its attributes and nested nodes. This is invaluable when you're trying to understand why your visitor isn't matching certain patterns or when you're exploring how Python represents unfamiliar syntax.

The `indent` parameter (added in Python 3.9) makes the output much more readable. For older Python versions, you can use `ast.dump(node)` without indentation, though it's harder to read. You can also use the `annotate_fields` parameter to show field names, which helps understand the structure.

In [2]:
# Complex code to analyze
code = """
class Handler:
    def process(self, data: List[str]) -> Optional[Result]:
        return self.transform(data) if data else None
"""

tree = ast.parse(code)

# Basic dump - hard to read
print("Basic dump:")
print(ast.dump(tree))  # Everything on one line!

Basic dump:
Module(body=[ClassDef(name='Handler', body=[FunctionDef(name='process', args=arguments(args=[arg(arg='self'), arg(arg='data', annotation=Subscript(value=Name(id='List', ctx=Load()), slice=Name(id='str', ctx=Load()), ctx=Load()))]), body=[Return(value=IfExp(test=Name(id='data', ctx=Load()), body=Call(func=Attribute(value=Name(id='self', ctx=Load()), attr='transform', ctx=Load()), args=[Name(id='data', ctx=Load())]), orelse=Constant(value=None)))], returns=Subscript(value=Name(id='Optional', ctx=Load()), slice=Name(id='Result', ctx=Load()), ctx=Load()))])])


In [3]:
# With indentation (Python 3.9+) - much better
print("\nIndented dump:")
print(ast.dump(tree, indent=2))


Indented dump:
Module(
  body=[
    ClassDef(
      name='Handler',
      body=[
        FunctionDef(
          name='process',
          args=arguments(
            args=[
              arg(arg='self'),
              arg(
                arg='data',
                annotation=Subscript(
                  value=Name(id='List', ctx=Load()),
                  slice=Name(id='str', ctx=Load()),
                  ctx=Load()))]),
          body=[
            Return(
              value=IfExp(
                test=Name(id='data', ctx=Load()),
                body=Call(
                  func=Attribute(
                    value=Name(id='self', ctx=Load()),
                    attr='transform',
                    ctx=Load()),
                  args=[
                    Name(id='data', ctx=Load())]),
                orelse=Constant(value=None)))],
          returns=Subscript(
            value=Name(id='Optional', ctx=Load()),
            slice=Name(id='Result', ctx=Load()),
            ctx=L

In [4]:
# Focusing on specific nodes
for node in ast.walk(tree):
    if isinstance(node, ast.FunctionDef):
        print("\nJust the function node:")
        print(ast.dump(node, indent=2))


Just the function node:
FunctionDef(
  name='process',
  args=arguments(
    args=[
      arg(arg='self'),
      arg(
        arg='data',
        annotation=Subscript(
          value=Name(id='List', ctx=Load()),
          slice=Name(id='str', ctx=Load()),
          ctx=Load()))]),
  body=[
    Return(
      value=IfExp(
        test=Name(id='data', ctx=Load()),
        body=Call(
          func=Attribute(
            value=Name(id='self', ctx=Load()),
            attr='transform',
            ctx=Load()),
          args=[
            Name(id='data', ctx=Load())]),
        orelse=Constant(value=None)))],
  returns=Subscript(
    value=Name(id='Optional', ctx=Load()),
    slice=Name(id='Result', ctx=Load()),
    ctx=Load()))


In [5]:
# Advanced: Custom filtering to show only what you care about
def dump_calls_only(node, level=0):
    """Recursively find and dump only Call nodes."""
    indent = "  " * level
    if isinstance(node, ast.Call):
        print(f"{indent}Call found:")
        print(ast.dump(node, indent=2))

    for child in ast.iter_child_nodes(node):
        dump_calls_only(child, level + 1)


# Practical debugging scenario
debug_code = "result = obj.method(x=10, y=20)"
debug_tree = ast.parse(debug_code)

print("\nDebugging a specific pattern:")
print(ast.dump(debug_tree, indent=2))


Debugging a specific pattern:
Module(
  body=[
    Assign(
      targets=[
        Name(id='result', ctx=Store())],
      value=Call(
        func=Attribute(
          value=Name(id='obj', ctx=Load()),
          attr='method',
          ctx=Load()),
        keywords=[
          keyword(
            arg='x',
            value=Constant(value=10)),
          keyword(
            arg='y',
            value=Constant(value=20))]))])


This shows us:
- The Call node structure
- How keyword arguments are represented
- The exact nesting of Attribute and Name nodes

In [6]:
# Pro tip: Create a helper function for debugging
def debug_ast(code_snippet):
    """Quick AST debugging helper."""
    print(f"Code: {code_snippet}")
    tree = ast.parse(code_snippet)
    print(ast.dump(tree, indent=2))
    return tree


# Use it for quick tests:
debug_ast("calc.add(1, 2)")

Code: calc.add(1, 2)
Module(
  body=[
    Expr(
      value=Call(
        func=Attribute(
          value=Name(id='calc', ctx=Load()),
          attr='add',
          ctx=Load()),
        args=[
          Constant(value=1),
          Constant(value=2)]))])


In [7]:
debug_ast("Calculator()")

Code: Calculator()
Module(
  body=[
    Expr(
      value=Call(
        func=Name(id='Calculator', ctx=Load())))])


In [8]:
debug_ast("x: int = 5")

Code: x: int = 5
Module(
  body=[
    AnnAssign(
      target=Name(id='x', ctx=Store()),
      annotation=Name(id='int', ctx=Load()),
      value=Constant(value=5),
      simple=1)])


## 18. Checking Node Types

Defensive programming is crucial when working with AST. Never assume a node has certain attributes - always check types first. This prevents crashes when encountering unexpected code patterns. The isinstance checks might seem verbose, but they make your code robust against edge cases and malformed input.

In [ ]:
# Example of defensive AST navigation
class SafeCallAnalyzer(ast.NodeVisitor):
    def visit_Call(self, node):
        print("Analyzing call:")

        # WRONG - Fragile approach that will crash:
        # method_name = node.func.attr  # AttributeError if func isn't Attribute!

        # RIGHT - Defensive approach with type checking:
        if isinstance(node.func, ast.Attribute):
            # Now safe to access .attr
            method_name = node.func.attr
            print(f"  Method/attribute call: .{method_name}()")

            # Continue drilling down safely
            if isinstance(node.func.value, ast.Name):
                obj_name = node.func.value.id
                print(f"    On object: {obj_name}")

                # Can now safely build the full call
                full_call = f"{obj_name}.{method_name}"
                print(f"    Full call: {full_call}()")

            elif isinstance(node.func.value, ast.Attribute):
                print("    On nested attribute access")
                # Handle module.class.method() patterns

            elif isinstance(node.func.value, ast.Call):
                print("    On call result (chained call)")
                # Handle get_obj().method() patterns

        elif isinstance(node.func, ast.Name):
            func_name = node.func.id
            print(f"  Simple function call: {func_name}()")

        elif isinstance(node.func, ast.Lambda):
            print("  Lambda call: (lambda ...)(...)")

        else:
            # Always have a fallback for unexpected patterns
            print(f"  Unexpected call pattern: {type(node.func).__name__}")

        # Safe argument checking
        if hasattr(node, "args"):  # Should always be true for Call
            print(f"  Args count: {len(node.args)}")

            # Safely analyze each argument
            for i, arg in enumerate(node.args):
                if isinstance(arg, ast.Constant):
                    print(f"    Arg {i}: constant {arg.value}")
                elif isinstance(arg, ast.Name):
                    print(f"    Arg {i}: variable {arg.id}")
                else:
                    print(f"    Arg {i}: {type(arg).__name__}")

        # Safe keyword argument checking
        if hasattr(node, "keywords"):
            for kw in node.keywords:
                if kw.arg:  # Named keyword argument
                    print(f"  Keyword arg: {kw.arg}=...")
                else:  # **kwargs expansion
                    print("  Keyword expansion: **...")

        self.generic_visit(node)

In [10]:
# Test with various call patterns
test_code = """
# Different call patterns to test robustness
simple_call()
obj.method()
module.sub.func()
get_handler().process()
(lambda x: x + 1)(5)
func(1, 2, x=3, y=4)
call(*args, **kwargs)
"""

tree = ast.parse(test_code)
analyzer = SafeCallAnalyzer()
analyzer.visit(tree)

Analyzing call:
  Simple function call: simple_call()
  Args count: 0
Analyzing call:
  Method/attribute call: .method()
    On object: obj
    Full call: obj.method()
  Args count: 0
Analyzing call:
  Method/attribute call: .func()
    On nested attribute access
  Args count: 0
Analyzing call:
  Method/attribute call: .process()
    On call result (chained call)
  Args count: 0
Analyzing call:
  Simple function call: get_handler()
  Args count: 0
Analyzing call:
  Lambda call: (lambda ...)(...)
  Args count: 1
    Arg 0: constant 5
Analyzing call:
  Simple function call: func()
  Args count: 2
    Arg 0: constant 1
    Arg 1: constant 2
  Keyword arg: x=...
  Keyword arg: y=...
Analyzing call:
  Simple function call: call()
  Args count: 1
    Arg 0: Starred
  Keyword expansion: **...


### Safe Attribute Chain Extraction

A common need is extracting full attribute chains like `module.submodule.function`. Here's a robust way to do it:

In [11]:
# Common pattern: Building a safe attribute chain extractor
def get_attribute_chain(node) -> Optional[str]:
    """Safely extract full attribute chain like 'a.b.c.d'."""
    parts = []

    current = node
    while isinstance(current, ast.Attribute):
        parts.append(current.attr)
        current = current.value

    # The base should be a Name
    if isinstance(current, ast.Name):
        parts.append(current.id)
        parts.reverse()
        return ".".join(parts)

    # If not, we have a complex base (like a call or subscript)
    return None  # Can't represent as simple chain


# Test the function
test_chain = ast.parse("module.sub.func").body[0].value
print(f"Attribute chain: {get_attribute_chain(test_chain)}")

Attribute chain: module.sub.func


In [ ]:
# Test with more complex cases
test_cases = [
    "simple_name",
    "obj.method",
    "module.submodule.function",
    "get_obj().method",  # This should return None
    "obj[0].method",     # This should return None
]

for case in test_cases:
    try:
        node = ast.parse(case).body[0].value
        result = get_attribute_chain(node)
        print(f"{case:20} -> {result}")
    except Exception as e:
        print(f"{case:20} -> Error: {e}")

simple_name          -> simple_name
obj.method           -> obj.method
module.submodule.function -> module.submodule.function
get_obj().method     -> None
obj[0].method        -> None


### Node Type Checking Patterns

Here are common defensive patterns you should use:

In [ ]:
def analyze_function_call(call_node: ast.Call) -> dict:
    """Example of comprehensive defensive analysis."""
    result = {
        "call_type": "unknown",
        "function_name": None,
        "object_name": None,
        "attribute_chain": None,
        "arg_count": 0,
        "has_kwargs": False,
        "has_starargs": False
    }

    # Safely analyze the function being called
    if isinstance(call_node.func, ast.Name):
        result["call_type"] = "function"
        result["function_name"] = call_node.func.id

    elif isinstance(call_node.func, ast.Attribute):
        result["call_type"] = "method"
        result["function_name"] = call_node.func.attr
        result["attribute_chain"] = get_attribute_chain(call_node.func)

        # Try to get the object name if it's simple
        if isinstance(call_node.func.value, ast.Name):
            result["object_name"] = call_node.func.value.id

    # Safely count arguments
    if hasattr(call_node, "args"):
        result["arg_count"] = len(call_node.args)

        # Check for *args
        for arg in call_node.args:
            if isinstance(arg, ast.Starred):
                result["has_starargs"] = True
                break

    # Check for **kwargs
    if hasattr(call_node, "keywords"):
        for kw in call_node.keywords:
            if kw.arg is None:  # **kwargs
                result["has_kwargs"] = True
                break

    return result


# Test the analyzer
test_calls = [
    "func()",
    "obj.method(1, 2)",
    "module.sub.func(x=1)",
    "call(*args, **kwargs)",
    "get_handler().process(data)"
]

for call in test_calls:
    tree = ast.parse(call)
    call_node = tree.body[0].value  # Extract the Call node
    analysis = analyze_function_call(call_node)
    print(f"\n{call}:")
    for key, value in analysis.items():
        print(f"  {key}: {value}")


func():
  call_type: function
  function_name: func
  object_name: None
  attribute_chain: None
  arg_count: 0
  has_kwargs: False
  has_starargs: False

obj.method(1, 2):
  call_type: method
  function_name: method
  object_name: obj
  attribute_chain: obj.method
  arg_count: 2
  has_kwargs: False
  has_starargs: False

module.sub.func(x=1):
  call_type: method
  function_name: func
  object_name: None
  attribute_chain: module.sub.func
  arg_count: 0
  has_kwargs: False
  has_starargs: False

call(*args, **kwargs):
  call_type: function
  function_name: call
  object_name: None
  attribute_chain: None
  arg_count: 1
  has_kwargs: True
  has_starargs: True

get_handler().process(data):
  call_type: method
  function_name: process
  object_name: None
  attribute_chain: None
  arg_count: 1
  has_kwargs: False
  has_starargs: False


### Advanced Debugging: Custom AST Dumper

Sometimes you want more control over how AST nodes are displayed:

In [ ]:
def custom_dump(node, indent=0, show_lineno=False):
    """Custom AST dumper with more control over output."""
    spaces = "  " * indent
    node_name = type(node).__name__

    # Add line number if requested and available
    lineno_str = ""
    if show_lineno and hasattr(node, 'lineno'):
        lineno_str = f" (line {node.lineno})"

    print(f"{spaces}{node_name}{lineno_str}")

    # Show important attributes
    if isinstance(node, ast.Name):
        print(f"{spaces}  id: {node.id}")
    elif isinstance(node, ast.Constant):
        print(f"{spaces}  value: {repr(node.value)}")
    elif isinstance(node, ast.FunctionDef):
        print(f"{spaces}  name: {node.name}")
    elif isinstance(node, ast.Attribute):
        print(f"{spaces}  attr: {node.attr}")

    # Recursively dump children
    for child in ast.iter_child_nodes(node):
        custom_dump(child, indent + 1, show_lineno)


# Test the custom dumper
sample_code = """
def example(x: int) -> str:
    result = obj.method(x)
    return str(result)
"""

tree = ast.parse(sample_code)
print("Custom dump with line numbers:")
custom_dump(tree, show_lineno=True)

Custom dump with line numbers:
Module
  FunctionDef (line 2)
    name: example
    arguments
      arg (line 2)
        Name (line 2)
          id: int
          Load
    Assign (line 3)
      Name (line 3)
        id: result
        Store
      Call (line 3)
        Attribute (line 3)
          attr: method
          Name (line 3)
            id: obj
            Load
          Load
        Name (line 3)
          id: x
          Load
    Return (line 4)
      Call (line 4)
        Name (line 4)
          id: str
          Load
        Name (line 4)
          id: result
          Load
    Name (line 2)
      id: str
      Load


## Key Takeaways

### Debugging Best Practices

1. **Always use `ast.dump()` with indentation** when debugging AST structures
2. **Create helper functions** like `debug_ast()` for quick testing
3. **Focus on specific node types** rather than dumping entire trees
4. **Use custom dumpers** when you need specific information highlighted

### Defensive Programming

1. **Always check node types** with `isinstance()` before accessing attributes
2. **Use `hasattr()`** to check for optional attributes
3. **Provide fallbacks** for unexpected node patterns
4. **Build utility functions** for common operations like attribute chain extraction

### Common Patterns

- Use `isinstance(node, ast.NodeType)` for type checking
- Use `hasattr(node, 'attribute')` for optional attributes  
- Build comprehensive analysis functions that return structured data
- Always call `self.generic_visit(node)` in visitor methods

## Next Steps

Continue with **Part 5: Advanced AST Patterns** to learn about:
- Complex visitor patterns
- AST transformation and modification
- Performance optimization techniques
- Real-world AST applications